In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter


import random

import time

import gc

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

warnings.filterwarnings("ignore")

In [ ]:
pip install einops

In [ ]:
from einops.layers.torch import Rearrange

In [ ]:
seed = 0

random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
DATA_PATH = "/kaggle/input/ventilator-pressure-prediction/"
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
# submission = pd.read_csv(DATA_PATH + "sample_submission.csv")

test_ids= test['id'].to_numpy()

In [ ]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    
    
#     df['area'] = df['time_step'] * df['u_in']
#     df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
#     df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['exponent']=(- df['time_step'])/(df['R']*df['C'])
    df['factor']=np.exp(df['exponent'])
    df['vf']=(df['u_in_cumsum']*df['R'])/df['factor']
    df['vt']=0
    df.loc[df['time_step'] != 0, 'vt']=df['area']/(df['C']*(1 - df['factor']))
    df['v']=df['vf']+df['vt']
    
#     df['R'] = df['R'].astype(str)
#     df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(train)
test = add_features(test)

targets = train[['pressure']].to_numpy().reshape(-1, 80)
train['w'] = 1 - train['u_out']
train.drop(['pressure', 'id', 'breath_id', 'u_out'], axis=1, inplace=True)
test['w'] = 1 - test['u_out']
test = test.drop(['id', 'breath_id', 'u_out'], axis=1)

r = train['R'].values.astype(np.float32).reshape(-1, 80, 1)
r = (r== 5)*1 + (r==20)*2 + (r==50)*3
c = train['C'].values.astype(np.float32).reshape(-1, 80, 1)
c = (c==10)*1 + (c==20)*2 + (c==50)*3

r_test = test['R'].values.astype(np.float32).reshape(-1, 80, 1)
r_test = (r_test== 5)*1 + (r_test==20)*2 + (r_test==50)*3
c_test = test['C'].values.astype(np.float32).reshape(-1, 80, 1)
c_test = (c_test==10)*1 + (c_test==20)*2 + (c_test==50)*3

RS = RobustScaler()
train = RS.fit_transform(train.iloc[:,2:])
test = RS.transform(test.iloc[:,2:])

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

train = np.concatenate((r, c, train), axis=2)
test = np.concatenate((r_test, c_test, test), axis=2)

y_test= np.zeros((test.shape[0],1))

In [ ]:
# General
verbose = 1
device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 0
save_weights = True

# Model
HIDDEN_SIZE = 256
NUM_LAYERS = 1
NUM_CLASSES = 1

# Main
INPUT_SIZE = train.shape[-1]-1
EPOCHS = 150
BATCH_SIZE = 512

# LR = 1e-3

In [ ]:
class VentilatorDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.w = data[:,:,-1]
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        df_sample = torch.tensor(self.data[idx, :, :-1],dtype= torch.float)
        df_target = torch.tensor(self.target[idx, :],dtype= torch.float)
        df_w = torch.tensor(self.data[idx, :, -1],dtype= torch.float)
        
        return df_sample, df_target, df_w

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size=INPUT_SIZE):
        super(RNNModel, self).__init__()        
        self.r_embed = nn.Embedding(4, 2)
        self.c_embed = nn.Embedding(4, 2)
        self.seq_embed = nn.Sequential(
            Rearrange('b l d -> b d l'),
            nn.Conv1d(2 + input_size, 32, kernel_size=5, padding=2, stride=1),
            Rearrange('b d l -> b l d'),
            nn.LayerNorm(32),
            nn.SiLU(),
            nn.Dropout(0.),
        )
        
        self.lstm1 = nn.LSTM(32, 400, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(2*400, 300, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(2*300, 200, batch_first=True, bidirectional=True)
        self.lstm4 = nn.LSTM(2*200, 100, batch_first=True, bidirectional=True)
        self.selu = nn.SELU()
#         self.dropout = nn.Dropout(p=0.5)
        
        self.head = nn.Sequential(
            nn.Linear(2*100, 50),
            nn.SiLU(),
        )
        self.pressure = nn.Linear(50, 1)
#         self.pressure_out = nn.Linear(50, 1)
        
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)

        for name, m in self.named_modules():
            if isinstance(m, nn.Linear):
                #print(name,m)
                nn.init.xavier_uniform_(m.weight.data)
                m.bias.data.fill_(0)

        
        
        
    def forward(self, x):
        batch_size = len(x)
        r = x[:,:,0].long()
        c = x[:,:,1].long()
        r = self.r_embed(r)
        c = self.c_embed(c)
        seq = torch.cat((r, c, x[:,:,2:]), 2)
        x = self.seq_embed(seq)
        
        x, _ = self.lstm1(x)
        x = self.selu(x)
        x, _ = self.lstm2(x)
        x = self.selu(x)
        x, _ = self.lstm3(x)
        x = self.selu(x)
        x, _ = self.lstm4(x)
        x = self.selu(x)
#         x = self.dropout(x)
        x = self.head(x)
        
        pressure = self.pressure(x).reshape(batch_size, 80)
#         pressure_out = self.pressure_out(x).reshape(batch_size, 80)
        return pressure #, pressure_out
        
#         self.rnn1 = nn.LSTM(input_size, 256, num_layers, batch_first=True, bidirectional=True)
# #         self.rnn2 = nn.LSTM(2*512, 256, num_layers, batch_first=True, bidirectional=True)
#         self.selu = nn.SELU()
#         self.fc1 = nn.Linear(2*256, num_classes)
    
    
    
#         nn.init.xavier_uniform_(self.rnn1.weight_ih_l0)
#         nn.init.orthogonal_(self.rnn1.weight_hh_l0)
#         nn.init.xavier_uniform_(self.rnn1.weight_ih_l0_reverse)
#         nn.init.orthogonal_(self.rnn1.weight_hh_l0_reverse)
        
# #         nn.init.xavier_uniform_(self.rnn2.weight_ih_l0)
# #         nn.init.orthogonal_(self.rnn2.weight_hh_l0)
# #         nn.init.xavier_uniform_(self.rnn2.weight_ih_l0_reverse)
# #         nn.init.orthogonal_(self.rnn2.weight_hh_l0_reverse)
        
#         nn.init.xavier_uniform_(self.fc1.weight)
                
#     def forward(self, x):
#         out, _ = self.rnn1(x)
# #         out = self.selu(out)
# #         out, _ = self.rnn2(out)
#         out = self.selu(out)
#         out = self.fc1(out)
                      
#         return out

In [ ]:
NUM_FOLDS = 10

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)
splits = kf.split(train, targets)

In [ ]:
class VentilatorLoss(nn.Module):
    def __call__(self, preds, y, w):
        mae = w * (y - preds).abs()
        mae = mae.sum(-1) / w.sum(-1)

        return mae

In [ ]:
predictions= np.zeros(test_ids.shape[0])

for i, (train_idx, val_idx) in enumerate(splits):
#     if i < 1:
    print(f"\n-------------   Fold {i + 1} / {5}  -------------\n")

    model = RNNModel().to(device)
    model.load_state_dict(torch.load('../input/weights-0171/ventilator0_val0171.pth', map_location=device))
#         model.load_state_dict(torch.load('../input/val-0221/ventilator0_val_0221.pth', map_location=device))
#         model.load_state_dict(torch.load('ventilator%d.pth' % i, map_location=device))
#         optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#     optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
    scheduler= torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=30, T_mult=1, eta_min=0, last_epoch=-1, verbose=True)
#     scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor= 0.5, patience= 5, verbose= True)
    criterion = nn.L1Loss()

    X_train, X_val= train[train_idx], train[val_idx]
    y_train, y_val= targets[train_idx], targets[val_idx]

    train_dataset = VentilatorDataset(data=X_train, target=y_train)
    val_dataset = VentilatorDataset(data= X_val, target= y_val)


    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    best_valid_loss = float('inf')

    loss_fct = VentilatorLoss()

    for epoch in range(EPOCHS):
        print(f"Epoch: {epoch+1}")
        model.train()
        start_time = time.time()

        sum_train_loss = 0
        for batch_idx, (X, y, w) in enumerate(train_loader):
            pred = model(X.to(device)).squeeze(-1) #(batch, seq, feature)

#                 train_loss =  w.to(device) * criterion(pred, y.to(device)) #for u_out == 1
            train_loss = loss_fct(pred, y.to(device), w.to(device)).mean()
            optimizer.zero_grad()
            train_loss.backward()
            sum_train_loss += train_loss.item()
            optimizer.step()
        epoch_train_loss = sum_train_loss / len(train_loader)
        print(f"epoch_train_loss={epoch_train_loss:.3f}")


        model.eval()
        sum_val_loss = 0

        with torch.no_grad():
            for batch_idx, (X, y, w) in enumerate(val_loader):
                pred = model(X.to(device)).squeeze(-1)

                val_loss = loss_fct(pred, y.to(device), w.to(device)).mean()
                sum_val_loss += val_loss.item()



        epoch_val_loss = sum_val_loss / len(val_loader)

        #save best model
        if (epoch_val_loss < best_valid_loss):
            best_valid_loss = epoch_val_loss
            ofilename = 'ventilator%d.pth' % i
            torch.save(model.state_dict(),  ofilename)

        scheduler.step(epoch_val_loss)
        
        print(f"LR={optimizer.param_groups[0]['lr']:.9f}")
        print(f"epoch_val_loss={epoch_val_loss:.3f}", f"   min_val_loss={best_valid_loss:.3f}")
        end_time = time.time()
        epoch_time = end_time - start_time
        print(f"epoch_time={epoch_time:.0f}")

    model.load_state_dict(torch.load('ventilator%d.pth' % i, map_location=device))
    model.eval()

    test_dataset = VentilatorDataset(data=test, target=y_test)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    preds = []
    with torch.no_grad():
        for batch_idx, (X, y, _) in enumerate(test_loader):
            pred = model(X.to(device)).squeeze(-1)
            preds.append(pred.detach().cpu().numpy())

    preds = np.concatenate(preds, 0)
    pred_test = preds.flatten()
    predictions += pred_test/1

In [ ]:
predictions= np.zeros(test_ids.shape[0])
model = RNNModel().to(device)
model.load_state_dict(torch.load('../input/weights-val0174/ventilator0_val0174.pth', map_location=device))
model.eval()

test_dataset = VentilatorDataset(data=test, target=y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

preds = []
with torch.no_grad():
    for batch_idx, (X, y, _) in enumerate(test_loader):
        pred = model(X.to(device)).squeeze(-1)
        preds.append(pred.detach().cpu().numpy())

preds = np.concatenate(preds, 0)
pred_test = preds.flatten()
predictions += pred_test/1

In [ ]:
submission = pd.DataFrame({'id': test_ids, 'pressure': predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
s = pd.read_csv('./submission.csv')

In [ ]:
s